# Capture Candidates Report (BMC Discovery)

This notebook reproduces the DisMAL `capture_candidates` report using the Tideway library only.
It lists unsupported devices with SNMP details, suitable for pattern capture. The CSV is saved
to `output_<target>/capture_candidates.csv`.

## Requirements

Uncomment the next cell to install dependencies in your environment if needed.

In [1]:
# %pip install -q tideway pandas pyyaml

import os, sys
from pathlib import Path
from typing import Any, Dict, List
import pandas as pd
import yaml
import tideway


## Configuration (from config.yaml)

Loads target, token/token_file, API version, and SSL flag; prepares output folder.

In [2]:
from pathlib import Path
import yaml

def load_config_params(
    start: Path,
    appliance_name: str = None,
    appliance_index: int = 0,
) -> dict:
    def _find_repo_root(start: Path) -> Path:
        for p in [start] + list(start.parents):
            if (p / 'config.yaml').exists():
                return p
        return start.parent

    repo_root = _find_repo_root(start)
    config_path = repo_root / 'config.yaml'

    with open(config_path, 'r') as fh:
        cfg = yaml.safe_load(fh) or {}

    apps = cfg.get('appliances') or []
    selected = None
    if isinstance(apps, list) and apps:
        if appliance_name:
            selected = next((a for a in apps if a.get('name') == appliance_name), None)
            if selected is None:
                raise ValueError(f"No appliance named '{appliance_name}' in config.yaml")
        else:
            try:
                selected = apps[int(appliance_index)]
            except Exception:
                selected = apps[0]

    target = ((selected or {}).get('target') or cfg.get('target') or '').strip()
    if not target:
        raise ValueError('config.yaml missing "target"')

    token = (((selected or {}).get('token') or cfg.get('token') or '').strip())
    token_file = (selected or {}).get('token_file') or cfg.get('token_file') or cfg.get('f_token')
    if not token and token_file:
        tf_path = Path(token_file)
        if not tf_path.is_absolute():
            tf_path = repo_root / tf_path
        with open(tf_path, 'r') as tf:
            token = tf.read().strip()
    if not token:
        raise ValueError('API token not found in config.yaml (token or token_file)')

    api_version = str((selected or {}).get('api_version') or cfg.get('api_version') or 'v1.14')
    verify_ssl = bool((selected or {}).get('verify_ssl', cfg.get('verify_ssl', True)))

    sanitized = target.replace('.', '_').replace(':', '_').replace('/', '_')
    output_dir = repo_root / f'output_{sanitized}'
    output_dir.mkdir(parents=True, exist_ok=True)

    return {
        "repo_root": repo_root,
        "config_path": config_path,
        "cfg": cfg,
        "selected": selected,
        "target": target,
        "token": token,
        "api_version": api_version,
        "verify_ssl": verify_ssl,
        "output_dir": output_dir,
    }

In [3]:
from pathlib import Path
import tideway

def init_appliance(appliance_name: str = "prod"):
    params = load_config_params(Path.cwd(), appliance_name=appliance_name)

    target = params["target"]
    api_version = params["api_version"]
    verify_ssl = params["verify_ssl"]
    output_dir = params["output_dir"]

    print('Base Host     :', target)
    print('API Version   :', api_version)
    print('Verify SSL    :', verify_ssl)
    print('Output folder :', output_dir)

    api_number = api_version.lstrip('v')

    app = tideway.appliance(target, params["token"], api_version=api_number, ssl_verify=verify_ssl)
    twsearch = app.data()

    try:
        about = app.api_about
        print('Appliance reachable:', about.status_code)
    except Exception as e:
        print('Warning: failed to contact appliance /api/about:', e)

    return {
        "params": params,
        "target": target,
        "app": app,
        "search": twsearch,
        "api_version":api_number,
        "output_dir":output_dir,
    }

# Initialise Instances

In [4]:
print("Initialise Prod:")
twprod = init_appliance("prod")

print("Initialise Dev:")
twdev = init_appliance("dev")

Initialise Prod:
Base Host     : geodisams-disc-itom.onbmc.com
API Version   : v1.14
Verify SSL    : True
Output folder : /Volumes/Hub/GitHub/Dismal/output_geodisams-disc-itom_onbmc_com
Appliance reachable: 200
Initialise Dev:
Base Host     : geodisams-disc-itom-dev.onbmc.com
API Version   : v1.14
Verify SSL    : True
Output folder : /Volumes/Hub/GitHub/Dismal/output_geodisams-disc-itom-dev_onbmc_com



## Run and Preview

Fetch the rows and preview the first few entries.

In [5]:
qry_capture = '''
search DiscoveryAccess where end_state = 'UnsupportedDevice' and _last_marker
traverse DiscoveryAccess:DiscoveryAccessResult:DiscoveryResult:DeviceInfo where sysobjectid
show
access_method as 'Access Method',
request_time as 'Request Time',
hostname as 'Hostname',
os as 'OS',
failure_reason as 'Failure Reason',
syscontact as 'Syscontact',
syslocation as 'Syslocation',
sysdescr as 'Sysdescr',
sysobjectid as 'Sysobject ID'
'''

In [7]:
# Run Query and Extract Results
def get_results(instance):
    results = instance['search'].search({'query': qry_capture}, format='object', limit=500)
    df = pd.DataFrame(results) if results else pd.DataFrame()
    if df.empty:
        # Provide headers if no rows returned
        df = pd.DataFrame(columns=['Access Method','Request Time','Hostname','OS','Failure Reason','Syscontact','Syslocation','Sysdescr','Sysobject ID'])
    df.insert(0, 'Discovery Instance', instance['target'])
    return df

df_prod = get_results(twprod)
print(twprod['target'])
display(df_prod.head(5))

df_dev = get_results(twdev)
print(twdev['target'])
display(df_dev.head(5))

geodisams-disc-itom.onbmc.com


,Discovery Instance,Access Method,Request Time,Hostname,OS,Failure Reason,Syscontact,Syslocation,Sysdescr,Sysobject ID
0,geodisams-disc-itom.onbmc.com,SNMP v2c,2025-09-23T01:03:31.459339+00:00,rmcard205,UPS SNMP Card,None,Administrator,Server Room,UPS SNMP Card,1.3.6.1.4.1.3808.1.1.1
1,geodisams-disc-itom.onbmc.com,SNMP v2c,2025-09-23T01:06:55.818895+00:00,rmcard205,UPS SNMP Card,None,Administrator,Server Room,UPS SNMP Card,1.3.6.1.4.1.3808.1.1.1
2,geodisams-disc-itom.onbmc.com,SNMP v2c,2025-09-23T01:21:00.252747+00:00,rmcard205,UPS SNMP Card,None,Administrator,Server Room,UPS SNMP Card,1.3.6.1.4.1.3808.1.1.1
3,geodisams-disc-itom.onbmc.com,SNMP v2c,2025-09-23T02:00:35.071989+00:00,rmcard205,UPS SNMP Card,None,Administrator,Server Room,UPS SNMP Card,1.3.6.1.4.1.3808.1.1.1
4,geodisams-disc-itom.onbmc.com,SNMP v2c,2025-09-23T02:03:27.169531+00:00,ams-ar-ff-cor1-fw-01,GEODIS AR FIREWALL,None,Mauro Moscuzza,Cordoba - Argentina,GEODIS AR FIREWALL,1.3.6.1.4.1.12356.101.1.351


geodisams-disc-itom-dev.onbmc.com


,Discovery Instance,Access Method,Request Time,Hostname,OS,Failure Reason,Syscontact,Syslocation,Sysdescr,Sysobject ID
0,geodisams-disc-itom-dev.onbmc.com,SNMP v2c,2025-09-23T07:06:31.349699+00:00,hpi7567b9,HP ETHERNET MULTI-ENVIRONMENT,None,None,None,HP ETHERNET MULTI-ENVIRONMENT,0.0
1,geodisams-disc-itom-dev.onbmc.com,SNMP v2c,2025-09-23T08:02:47.336269+00:00,hpidcfaaa,HP ETHERNET MULTI-ENVIRONMENT,None,None,None,HP ETHERNET MULTI-ENVIRONMENT,0.0
2,geodisams-disc-itom-dev.onbmc.com,SNMP v2c,2025-09-23T10:01:55.336269+00:00,hpi295e8c,HP ETHERNET MULTI-ENVIRONMENT,None,None,None,HP ETHERNET MULTI-ENVIRONMENT,0.0
3,geodisams-disc-itom-dev.onbmc.com,SNMP v2c,2025-09-23T12:01:34.251091+00:00,p-4280ac,Integrated PrintNet Enterprise Version 2.8.6 3...,None,None,None,Integrated PrintNet Enterprise Version 2.8.6 3...,1.3.6.1.4.1.2021.250.255
4,geodisams-disc-itom-dev.onbmc.com,SNMP v2c,2025-09-23T13:03:44.973930+00:00,hpie4931e,HP ETHERNET MULTI-ENVIRONMENT,None,None,None,HP ETHERNET MULTI-ENVIRONMENT,0.0


## Save to CSV

Writes `capture_candidates.csv` to the standard output folder.

In [8]:
output_prod = str(twprod['output_dir'] / 'capture_candidates.csv')
df_prod.to_csv(output_prod, index=False)
output_dev = str(twdev['output_dir'] / 'capture_candidates.csv')
df_dev.to_csv(output_dev, index=False)

print(f'Saved to {output_prod}')
print(f'Saved to {output_dev}')


Saved to /Volumes/Hub/GitHub/Dismal/output_geodisams-disc-itom_onbmc_com/capture_candidates.csv
Saved to /Volumes/Hub/GitHub/Dismal/output_geodisams-disc-itom-dev_onbmc_com/capture_candidates.csv


End